In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.9/314.9 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 91.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for to

In [2]:
!pip install -U pip setuptools wheel

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:0000:010:01
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [3]:
!pip install tokenizers --only-binary=:all:


In [4]:
!pip install trl peft transformers accelerate


In [5]:
import os 
import torch 

In [6]:
from datasets import load_dataset
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    pipeline,
    logging,
)

from peft import LoraConfig,PeftModel
from trl import SFTTrainer


## **IN CASE of LLaMA 2, the Following Prompt Template Is Used for the Chat Models**

### **Prompt Components**

- **System Prompt (optional)**  
  Used to guide the model’s behavior and rules.

- **User Prompt (required)**  
  Provides the instruction or question to the model.

- **Model Answer (required)**  
  The response generated by the model.

---

### **LLaMA 2 Chat Prompt Template**

```text
<s> [INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model answer </s>

^
|
|
<s>start-of-sequence token. 
[INST] ... [/INST] -This wraps the instruction block.
        Inside [INST] you usually place:
            The system prompt
            The user prompt



**WE WILL REFORMAT OUR INSTRUCTION DATASET TO FOLLOW LLama 2 template**

using this link 
https://huggingface.co/datasets/timdettmers/openassistant-guanaco/viewer/default/train?row=0&views%5B%5D=train

Note= You don't need to follow a specific prompt template if you're using the base Llama 2 model instead of chat version. 

**How to Fine Tune LLama2**

1-Free google colab offers a 15gb GPU(LIMITED RESOURCES -> Barely enough to store Llama-2 2-7b's weights)
2-We also need to consider the overhead due to optimizer states,gradients,and forward activations
3-Full fine-tuning is not possible here: we need parameter-efficent fine-tuning(PEFT) tuning like LoRA or QLoRA.
4-TO drastically reduce the VRAM usage, we must finw-tune the model in 4-bit precison,which is why we'll use QLoRA here.

Now, We will 
        1)Load a llama-2-7b-chat-hf model (chat model)
        2)Train it on the mlabonne/guanaco-llama-2-1k(1,000 samples), which will produce our fine-tuned Llama-2-7b-chat-finetune

QLoRA will use a rank of 64 with a scaling parameter of 16. We'll load the Llama 2 model directly in 4-bit precision using the NF4 type and train it for one epoch

In [7]:
# The Model that you want to Train from the HUGGING_FACE hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

#the instruction dataset to use 
dataset_name ="mlabonne/guanaco-llama2-1k"

new_model = "Llama-2-7b-chat-finetune"

##################################################
# QLoRA parameters
##################################################

#LoRA attention parameters(RANK OF THE MATRIX)
lora_r = 64

#Alpha parameter for LoRA scaling(scaling parameter)
lora_alpha = 16

#Dropout Probablity for LoRA layers 
lora_dropout = 0.1

############################################################
# bitsandbytes parameters
############################################################

#Activating 4-bit precision base model loading
use_4bit = True

#Compute dtype for 4bit base models
bnb_4bit_compute_dtype = "float16"

#Quantization type(fp4 or nf4) nf4 
bnb_4bit_quant_type = "nf4"

#Activate nested quantization for 4-bit base models(double quantization)
use_nested_quant = False

###############################################################
# TrainingArguments parameters
###############################################################

#OutPut directory where the model predicition and checkpoints will be stored 
output_dir = "./results"

#Number of training epochs
num_train_epochs = 1

#Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

#Batch size GPU for training 
per_device_train_batch_size = 4 

#Batch size per GPU for traning
per_device_train_batch_size = 4

#Batch size per GPU for evaluation 
per_device_eval_batch_size = 4

#Number of update steps to accmulated the gradients for 
gradient_accmulation_steps = 1

#Enable gradient checkpointing 
gradient_checkpointing = True

#Maximum gradient normal (graphient clipping)
max_graph_norm =0.3

#Inital learning rate(AdamW optimizer)
learning_rate = 2e-4

#weight decay to apply to all layers except bias/LayersNorm weights 
weight_decay =0.001

#Optimizer to use 
optim = "paged_adamw_32bit"

#Learning rate schedule 
lr_scheduler_type = "cosine"

#Number of Traning steps(overrides  num_train_epochs)
max_steps = -1

#Ratio of steps for linear warmup(from 0 to learning rate)
warmup_ratio = 0.03

#group sequence into batches with same length 
#saves memory and speeds up training considerably
group_by_length = True 

#Save checkpoint every X update Steps 
save_steps = 0 

#Log every X updates steps 
logging_steps  = 25 


######################################################
#SFT parameters  SuperVised training
######################################################

#Maximum sequence length to use 
max_seq_length = None

#Pack multipule short example in the sample input sequence to increase efficency 
packing = False

#Load the entire model on the GPU 0 
device_map = {"":0}




STEP 4 : Load Everything and start the fine-tuning process 

        1. First of all, we want to load the dataset we defined. Here,out dataset is already preprocessed but, usually, this is where you would reformat the prompt, filter out bad text, combine multipule datasets,etc.
        
        2.Then, we're configuring bitsandbytes for 4-bit quantization.
        
        3.Next, we're loading the LLama2 model in 4-bit precision on a GPU with the corresponding tokenizer.
        
        4.Finally, we're loading configuration for QLoRA, regular training parameters,and Passing everything to the SFTTrainer. The training can finally start.  

In [8]:
#Load dataSet(you can process it here )
dataset = load_dataset(dataset_name , split= "train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

1.Checks local cache
2.AT ~/.cache/huggingface/datasets/
3.if not found:
        i. Downloads the dataset files
        ii.Downloads metadata
4.Processes the dataset:
        i.Parses files.
        ii.builds an efficenet arrow table  
Returns Dataset object


```text
model_name = "NousResearch/Llama-2-7b-chat-hf"
this is from hugging face in the format <organization-or-user>/<repo-name>

if this was a local model it would have been 
model_name = "./models/llama2"  

and same thing for dataset 
dataset_name = "mlabonne/guanaco-llama2-1k"


ARROW TABLE :- if you forget what is this https://www.youtube.com/watch?v=hVW7kflCyIQ here is the video 


Split has three options:- it means which split you want? do you want training split or you want test split or you want validation split.....; for now we want training split so "train"

In [9]:
#Load tokenizer anf Model with QLORA configuration

compute_dtype = getattr(torch,bnb_4bit_compute_dtype)

```text
It converts strings → torch dtypes
torch datatypes are:
    fp16, fp32 and all

layman  
    “Give me the attribute of object whose name is this string.”


    returns one of these objects
        torch.float16,
        torch.bfloat16,
        torch.float32    
it is equivalent to 
compute_dtype = torch.float16



In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
    )

**BitsandBytesConfig() is a quantization confihuration object.**
```text
the above piece of code explains how the model weights are loaded in 4-bit weight percsion
     load_in_4bit, 
        if this parameter is true than the model weights are loaded in 4-bit precision
        if this happens there is Massive VRAM saving 
    bnb_4bit_quant_type,
        this is specifies how the 4-bit numbers are stored (nf4)
    bnb_4bit_compute_dtype:
        this is the datatype of the weights of the model in this case float16
    bnb_4bit_use_double_quant:
        this is the part where it quantizes the quanitzed value,
        but not exactly, it quanitized scale factor

In [18]:
#check GPU compalibility with bfloat16

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability() #returns a tuple (major, minor)
    if major >=8:
        print("=" * 80 )
        print("Your GPU support bfloat16: accelerate traning with bf16=true ")  
        print("="* 80 )

AssertionError: Torch not compiled with CUDA enabled

In [17]:
#Load Base Model 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
#LOAD LLaMa tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_taken

In [13]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 35.8 MB/s  0:00:01m0:00:0100:01


In [16]:
!pip install -U transformers bitsandbytes

In [ ]:
    import torch
    print(torch.cuda.is_available())
    print(torch.cuda.get_device_name(0))

False


AssertionError: Torch not compiled with CUDA enabled

In [20]:
# Safe CUDA info check (avoids AssertionError when PyTorch not built with CUDA)
available = torch.cuda.is_available()
print("torch.cuda.is_available():", available)

if not available:
    print("CUDA not available or PyTorch not compiled with CUDA.")
    print("torch.__version__:", torch.__version__)
    print("torch.version.cuda:", torch.version.cuda)
else:
    try:
        count = torch.cuda.device_count()
        print("CUDA device count:", count)
        for i in range(count):
            name = torch.cuda.get_device_name(i)
            props = torch.cuda.get_device_properties(i)
            print(f"Device {i}: {name} | total_memory={props.total_memory/1024**3:.2f} GB | major={props.major} minor={props.minor}")
    except Exception as e:
        print("Error querying CUDA devices:", repr(e))

torch.cuda.is_available(): False
CUDA not available or PyTorch not compiled with CUDA.
torch.__version__: 2.9.0+cpu
torch.version.cuda: None
